In [8]:
import os
import importlib
import logging
importlib.reload(logging)
import framework
importlib.reload(framework)
import infer_bert_classifier
importlib.reload(infer_bert_classifier)
import bert_utils
importlib.reload(bert_utils)
from framework import *
import pandas as pd
# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)

ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Set up the task details. This notebook handles NER (for labeling person and company names)

In [9]:
DATASET = 'w2' # supports w2 and resume
TASK_CONFIG = {
    'task': 'ner',
    'num_labels': 3,
    'labels_dict': {'person' : 0, 'org' : 1, 'none': 2}
}

task = Task_NER(TASK_CONFIG)

In [10]:
W2_DATA = [
   '/Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records'
]
W2_GOLDEN = [
   '/Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv'
]

GOLDEN_CONFIG = {
    'path': W2_GOLDEN,
    'is_local': True,
    'index_field_name':'filename',
    'file_type': 'csv',
    'identifier': 'file'
}
DATASET_CONFIG = {
    'path': W2_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': True
}

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/w2-instabase/flow/s2_map_records
INFO:root:142 files loaded
INFO:root:Loading goldens from /Users/ahsaasbajaj/Documents/Data/w2-instabase/golden/goldens.csv
INFO:root:Total files Goldens: (154, 25)
INFO:root:Total files found in the source with unique index: (142, 25)
INFO:root:Processing 142 IBOCR files to txt


Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [11]:
DATASET = 'w2' # supports w2 and resume
TASK_CONFIG = {
    'task': 'ner',
    'num_labels': 3,
    'labels_dict': {'person' : 0, 'org' : 1, 'none': 2}
}

task = Task_NER(TASK_CONFIG)

In [12]:
W2_REFINER_RESULT_PATH = '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/refiner_results/w2.ibocr'
RESUME_REFINER_RESULT_PATH = '/Users/ahsaasbajaj/Documents/Code/ner-hf/sequence-classification/refiner_results/resume.ibocr'

DATA_ARGS = {
    'task': task,
    'dataset': data,
    'candidates_fields': {
        'person':'employee_name',
        'org':'employer_name'
    }
}
TRAINING_ARGS = {
    'model_file_or_path' : W2_REFINER_RESULT_PATH,
    'num_labels': TASK_CONFIG['num_labels'],
}

MODELS_TO_EVAL = {
    'models': ['names_vontell', 'names_token_matcher'],
    'spacy_models': ['names_spacy', 'org_spacy'],

    'person_name_models': ['names_vontell', 'names_token_matcher', 'names_spacy'],
    'org_name_models': ['org_spacy'],
}

Load ouputs from refiner flow (after step 4 producing single out.ibocr) and get extractions and metrics (Recall, Precision, F1) by comparing with goldens

In [13]:
model = Refiner(DATA_ARGS, TRAINING_ARGS, MODELS_TO_EVAL)
results = model.analyze_results()

INFO:root:
Person Name Scores
INFO:root:For model names_vontell, recall: 0.7465, precision: 0.4180, F1: 0.5359 
INFO:root:For model names_token_matcher, recall: 0.6549, precision: 0.4602, F1: 0.5405 
INFO:root:For model names_spacy, recall: 0.0915, precision: 0.0034, F1: 0.0066 
INFO:root:
Org Name Scores
INFO:root:For model org_spacy, recall: 0.0775, precision: 0.0012, F1: 0.0023 


Print extractions

In [14]:
# model.demo(results)